In [4]:
from vortice.config_global import SiteConfig, GhgDataset
from vortice.config_instruments import Anemometer3DConfig, GasAnalyzerConfig
from vortice.config_data_fields import (
    Wind3dUField,
    Wind3dVField,
    Wind3dWField,
    PressureField,
    TemperatureField,
    GhgDiagnosticField,
    H2OField,
    CO2Field,
    CH4Field,
    CH4MoleField,
)

cachoeira = SiteConfig(
    name="CS", lat=-30.446667, lon=-53.194444, alt=73.0, canopy_height=0.1
)

csat_3d = Anemometer3DConfig(
    name="CSAT3D", model="CSAT3_1", height=3.0, north_offset=90.0
)

li_7500a = GasAnalyzerConfig(name="LI7500A", model="li7500a")
li_7700 = GasAnalyzerConfig(name="LI7700", model="li7700")

fields = [
    Wind3dUField(name="u", instrument=csat_3d, unit="m/s"),
    Wind3dVField(name="v", instrument=csat_3d, unit="m/s"),
    Wind3dWField(name="w", instrument=csat_3d, unit="m/s"),
    TemperatureField(name="TS", instrument=csat_3d, unit="c"),
    GhgDiagnosticField(name="diag_CSAT", instrument=csat_3d),
    GhgDiagnosticField(name="Diag_LI7500A", instrument=li_7500a),
    PressureField(name="Pressao_LI7500A", instrument=li_7500a, unit="kPa"),
    TemperatureField(name="Temp_LI7500A", instrument=li_7500a, unit="c"),
    H2OField(name="H2O", instrument=li_7500a, unit="mmol/m^3"),
    CO2Field(name="CO2", instrument=li_7500a, unit="mmol/m^3"),
    GhgDiagnosticField(name="Diag_LI7700", instrument=li_7700),
    PressureField(name="Pressao_LI7700", instrument=li_7700, unit="kPa"),
    TemperatureField(name="Temp_LI7700", instrument=li_7700, unit="c"),
    CH4Field(name="CH4", instrument=li_7700, unit="mmol/m^3"),
    CH4MoleField(name="CH4_mole_fraction", instrument=li_7700, unit="umol/mol"),
    GhgDiagnosticField(name="RSSI", instrument=li_7700),
    GhgDiagnosticField(name="Drop_rate", instrument=li_7700),
]

CS130 = GhgDataset(
    name="CS130",
    site=cachoeira,
    columns=fields,
    time_column="TIMESTAMP",
    tz="UTC",
    freq=10,
)

In [5]:
from vortice.config import FluxDataFrame
import duckdb

con = duckdb.connect(database=":memory:", read_only=False)

query = """
SELECT * EXCLUDE(month, year, site) 
FROM parquet_scan('../../dados_lumet/site=CS130/**/*.parquet', hive_partitioning = true) 
WHERE year = 2023 
AND month = 12 
AND TIMESTAMP > '2023-12-01 12:00:00' 
AND TIMESTAMP <= '2023-12-01 13:00:00' 
ORDER BY TIMESTAMP;
"""
result = con.execute(query)

df_hf = (
    result.pl()
    .sort("TIMESTAMP")
    .upsample(time_column="TIMESTAMP", every="100ms", maintain_order=True)
)

In [6]:
df_hf.flux.config(CS130)